# Basic Deep Learning Face Recogntion


In [23]:
#Our Keras imports
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os, os.path


# DATASETS FROM FOLDERS

In [22]:
img_rows, img_cols = 48, 48
batch_size = 16

train_data_dir = 'C:/Users/Predator/PycharmProjects/AwFR/Face Recognition/faces/train'
validation_data_dir = 'C:/Users/Predator/PycharmProjects/AwFR/Face Recognition/faces/validation'

#data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

train_samples = sum(len(files) for a, b, files in os.walk(r'C:/Users/Predator/PycharmProjects/AwFR/Face Recognition/faces/train'))

validation_samples = sum(len(files) for a, b, files in os.walk(r'C:/Users/Predator/PycharmProjects/AwFR/Face Recognition/faces/validation'))

Found 2948 images belonging to 5 classes.
Found 996 images belonging to 5 classes.


# Class Labels

In [3]:
class_labels = train_generator.class_indices
class_labels = {i: j for j, i in class_labels.items()}
classes = list(class_labels.values())
num_classes = len(class_labels)
class_labels

{0: 'Chandler', 1: 'DIKESH', 2: 'Joey', 3: 'Pheobe', 4: 'Rachel'}

# Creating a simple VGG based model for Face Recognition

In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = 'same', kernel_initializer="he_normal",
                 input_shape = (img_rows, img_cols, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding = "same", kernel_initializer="he_normal", 
                 input_shape = (img_rows, img_cols, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #2: second CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #3: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #4: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #5: first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #6: second set of FC => RELU layers
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


# Block #7: softmax classifier
model.add(Dense(num_classes, kernel_initializer="he_normal"))
model.add(Activation("softmax"))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

# Training our Model

In [5]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("C:/Users/Predator/PycharmProjects/AwFR/Trained models/face_recognition_friends_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.01),
              metrics = ['accuracy'])

epochs = 50

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
184/184 [==============================] - ETA: 0s - loss: 1.2346 - accuracy: 0.5392
Epoch 00001: val_loss improved from inf to 2.23563, saving model to C:/Users/Predator/PycharmProjects/AwFR/Trained models\face_recognition_friends_vgg.h5
184/184 [==============================] - 47s 255ms/step - loss: 1.2346 - accuracy: 0.5392 - val_loss: 2.2356 - val_accuracy: 0.4325
Epoch 2/50
184/184 [==============================] - ETA: 0s - loss: 0.7493 - accuracy: 0.7176
Epoch 00002: val_loss improved from 2.23563 to 0.88990, saving model to C:/Users/Predator/PycharmProjects/AwFR/Trained models\face_recognition_friends_vgg.h5
184/184 [==============================] - 48s 258ms/step - loss: 0.7493 - accuracy: 0.7176 - val_loss: 0.8899 - val_accuracy: 0.7933
Epoch 3/50
184/184 [==============================] - ETA: 0s - loss: 0.6711 - accuracy: 0.7544
Epoch 00003: val_loss did not improve from 0.88990
184/1

## Saving our Model

In [25]:
model.save("C:/Users/Predator/PycharmProjects/AwFR/face_recognition_friends_vgg.h5")
print("Model Saved")

Model Saved


# Loading our model

In [26]:
from keras.models import load_model

classifier = load_model('C:/Users/Predator/PycharmProjects/AwFR/face_recognition_friends_vgg.h5')

### Testing our model on some real video

In [27]:
from os import listdir
from os.path import isfile, join
import os
import cv2
import numpy as np
import dlib 


face_classes = class_labels

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
margin = 0.2
# load model and weights
img_size = 64

detector = dlib.get_frontal_face_detector()

cap = cv2.VideoCapture('C:/Users/Predator/PycharmProjects/AwFR/Face Recognition/DIKU.mp4')

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    preprocessed_faces = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            #faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face = cv2.resize(face, (48, 48), interpolation = cv2.INTER_AREA)
            face = face.astype("float") / 255.0
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)
            preprocessed_faces.append(face)
 
        face_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces[i])[0]
            face_labels.append(face_classes[preds.argmax()])
        
        # draw results
        for i, d in enumerate(detected):
            label = "{}".format(face_labels[i])
            draw_label(frame, (d.left(), d.top()), label)

    cv2.imshow("Student identified", frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()      